In [ ]:
import pandas as pd
import altair as alt
import geopandas as gpd

In [ ]:
world = gpd.read_file('world-countries.json')

In [ ]:
pop_data = pd.read_csv('population_prospects.csv')

In [ ]:
world.head()

In [ ]:
pop_data.head()

In [ ]:
pop_data['variant'].unique()

In [ ]:
pop_data_2000 = pop_data[(pop_data['variant'] =='Estimates') & (pop_data['year'] ==2000) ]

In [ ]:
pop_data_2000.head()

In [ ]:
pop_data_2100 = pop_data[(pop_data['variant'] =='Medium variant') & (pop_data['year'] ==2100) ]

In [ ]:
pop_data_2100.head()

In [ ]:
pop_data_needed = pop_data_2000.copy()

In [ ]:
pop_data_needed.rename(columns={"population": "pop_2000"}, inplace=True)
pop_data_needed = pop_data_needed.merge(pop_data_2100[["country_code", "population"]], on= "country_code")
pop_data_needed.rename(columns={"population": "pop_2100"}, inplace=True)

In [ ]:
pop_data_needed.loc[pop_data_needed["country"] == "Russian Federation", "country"] = "Russia"

In [ ]:
pop_data_needed.head()

In [ ]:
pop_data_needed["2100gt2000"] = pop_data_needed["pop_2100"] > pop_data_needed["pop_2000"]

In [ ]:
pop_data_needed.head()

**В яких країнах світу населення у 2100 році буде більшим, ніж було у 2000 році, а в яких меншим?**

Я вибрав кольорову шкалу із 2 кольорів, які чітко відповідають на 1 питання - чи більше буде населення в країні 2100 році порівняно з 2000. Також я додав 3 колір щоб відмітити країни, для яких інформація з датасеру не була знайдена (скоріше бо назви країн різняться). Також я додав tooltip зі значенням популяції у 2100 та 2000 роках, щоб можна було одразу оцінити приблизно наскільки популяція різниться.

In [ ]:
alt.Chart(world).transform_lookup(
    lookup = 'name',
    from_ = alt.LookupData(data = pop_data_needed, 
                           key = 'country',
                           fields=['2100gt2000', 'country', 'pop_2000', 'pop_2100'])
).project(type = 'equalEarth'
).mark_geoshape().encode(
    color = alt.Color('2100gt2000:N'),
    tooltip = [alt.Tooltip('2100gt2000:N'),
    alt.Tooltip('pop_2000:N'),
    alt.Tooltip('pop_2100:N')]
).properties(width = 850, height = 500, background = '#F9F9F9', padding = 25)

**На скільки відсотків збільшиться або зменшиться населення кожної країни у 2100 році у порівнянні з 2000 роком?**

Я вибрав кольорову шкалу з проміжками між кольорами в 20 одиниць, бо хотів підкреслити ксть країн з прогнозованим відсотковим збільшенням популяції близьким до 0.

In [ ]:
pop_data_needed["2100_2000_perc_delta"] = 100 * (pop_data_needed["pop_2100"] - pop_data_needed["pop_2000"]) / pop_data_needed["pop_2000"]

In [ ]:
alt.Chart(world).transform_lookup(
    lookup = 'name',
    from_ = alt.LookupData(data = pop_data_needed, 
                           key = 'country',
                           fields=['2100_2000_perc_delta', 'country'])
).project(type = 'equalEarth'
).mark_geoshape().encode(
    color = alt.Color('2100_2000_perc_delta:Q', bin = alt.Bin(step = 20)),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('2100_2000_perc_delta:N',format = '<1d')
    ]
).properties(width = 850, height = 500, background = '#F9F9F9', padding = 25)

**Яким буде населення країн світу у 2100 році?**

Я вибрав pinkyellowgreen, бо вона краще відображає контраст між країнами з високим населенням та низьким. Попередня кольорова шкала була більш плавною

In [ ]:
alt.Chart(world).transform_lookup(
    lookup = 'name',
    from_ = alt.LookupData(data = pop_data_needed, 
                           key = 'country',
                           fields=['pop_2100', 'country'])
).project(type = 'equalEarth'
).mark_geoshape().encode(
    color = alt.Color('pop_2100:Q', scale = alt.Scale(type = 'quantize', scheme = 'pinkyellowgreen')),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('pop_2100:N',format = '<1d')
    ]
).properties(width = 850, height = 500, background = '#F9F9F9', padding = 25)